In [ ]:
#Importing necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import time

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,  precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from openpyxl import load_workbook

In [ ]:
#Reading the excel-document into a df
df = pd.read_excel('dataset_name.xlsx', Sheet_name = 'Sheet_name') 

#Choosing which columns to keep and saving them in a new df
columns_to_keep = ['Column 1', 'Column 2', 'Column 3', 'Column 4', 'Column 5', '...']
df2 = df[columns_to_keep].copy()

In [ ]:
#Splitting df into two based on hole ID
df2.loc[:, 'Hole_id_stat_str'] = df2['Hole_id_stat'].astype(str) #Making sure the IDs are strings

#Defining the unseen column(s)
columns_unseen = ['Column']
#Splitting into train and unseen data based on column
df_train = df2[~df2['Hole_id_stat_str'].isin(columns_unseen)] 
df_unseen = df2[df2['Hole_id_stat_str'].isin(columns_unseen)] 


In [ ]:
#Defining which columns to keep av model input
X = df_train.drop(columns = ['Column 1', 'Column 2', 'Column 3'])

#Defining which columns are the model outputs
y = df_train['Column 4']

#Defining inputs and outputs for unseen dataset
X_unseen = df_unseen.drop(columns = ['Column 1', 'Column 2', 'Column 3')
y_unseen = df_unseen['Column 4'] 

In [ ]:
#Setting up K-Fold cross-validation
k_fold = KFold(n_splits=5, random_state=66, shuffle=True)

# Initializing a Random Forest Classifier
RF = RandomForestClassifier(n_estimators=40, max_depth=7, random_state=40, class_weight='balanced')

# Creating lists to store performance metrics for each fold
accuracies = []
precisions = []
recalls = []
f1_scores = []

#Looping over each fold in the cross-validation
for train_index, test_index in k_fold.split(X):
    #Splitting the dataset into training and test sets based on the current fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    #Printing class distribution in the training set
    print(f"Train Class Distribution: {Counter(y_train)}")

    #Scaling features using StandardScaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    #Training the Random Forest model on the scaled training data
    RF.fit(X_train_scaled, y_train)

    #Making predictions on the scaled test data
    pred_test = RF.predict(X_test_scaled)

    #Evaluating and storing performance metrics for this fold
    accuracies.append(accuracy_score(y_test, pred_test))
    precisions.append(precision_score(y_test, pred_test, average='weighted', zero_division=0))
    recalls.append(recall_score(y_test, pred_test, average='weighted'))
    f1_scores.append(f1_score(y_test, pred_test, average='weighted'))

#Printing the average performance across all folds
print(f"Average Accuracy: {sum(accuracies)/len(accuracies):.4f}")
print(f"Average Precision: {sum(precisions)/len(precisions):.4f}")
print(f"Average Recall: {sum(recalls)/len(recalls):.4f}")
print(f"Average F1 Score: {sum(f1_scores)/len(f1_scores):.4f}")

In [ ]:
#Defining a grid of hyperparameters to search over

parameters_grid = {
    'n_estimators': [50, 75, 100],        # Number of trees in the forest
    'max_features': ['sqrt', 'log2'],     # Strategy to select the number of features for the best split
    'max_depth': [10, 15, 20],            # Maximum depth of each tree
    'min_samples_split': [10, 12, 15],    # Minimum number of samples required to split an internal node
    'min_samples_leaf': [2, 3, 5, 7],     # Minimum number of samples required to be at a leaf node
    'criterion': ['gini', 'entropy'],     # Function to measure the quality of a split
    'random_state': [66]                  # Ensures reproducibility of results
}

#Setting up GridSearchCV using the defined parameter grid
CV_RF = GridSearchCV(estimator=RF, param_grid=parameters_grid, cv=k_fold)

#Fitting the grid search on the scaled training data
CV_RF.fit(X_train_scaled, y_train)

#Printing the best hyperparameter combination found during the search
print('Best parameters: ', CV_RF.best_params_)


In [ ]:
#Saving the best hyperparameters found 
best_params = CV_RF.best_params_

#Defining class weighting to handle imbalanced classes
class_weights = 'balanced'

#Initializing a new Random Forest model using the best parameters and class weights
RF_best = RandomForestClassifier(**best_params, class_weight=class_weights)

#Training the model on the scaled training data
RF_best.fit(X_train_scaled, y_train)

#Making predictions using the trained model on the train data
pred_train = RF_best.predict(X_train_scaled)

#Making predictions using the trained model on the test data
pred_test = RF_best.predict(X_test_scaled)

In [ ]:
# Evaluate model performance on the training data
acc_train = accuracy_score(y_train, pred_train)
precision_train = precision_score(y_train, pred_train, average='weighted')
recall_train = recall_score(y_train, pred_train, average='weighted')
f1_train = f1_score(y_train, pred_train, average='weighted')

#Printing performance metrics
print(f'Accuracy: {acc_train:.4f}')
print(f'Precision: {precision_train:.4f}')
print(f'Recall: {recall_train:.4f}')
print(f'F1 Score: {f1_train:.4f}')

#Creating and displaying a confusion matrix
cm = confusion_matrix(y_train, pred_train)

#Generating class labels based on classes in the training data
class_labels = [f"Class {i}" for i in np.unique(y_train)]

#Visualizing the confusion matrix 
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_labels, yticklabels=class_labels)

plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
# Evaluate model performance on the test data
# Get all unique class labels from the full dataset
all_labels = np.unique(y)

#Computing evaluation metrics for the test data
acc_test = accuracy_score(y_test, pred_test)                         
precision_test = precision_score(y_test, pred_test, average='weighted')  
recall_test = recall_score(y_test, pred_test, average='weighted', zero_division=0.0)  
f1_test = f1_score(y_test, pred_test, average='weighted')             

#Printing the test data performance metrics
print(f'Accuracy: {acc_test:.4f}')
print(f'Precision: {precision_test:.4f}')
print(f'Recall: {recall_test:.4f}')
print(f'F1 Score: {f1_test:.4f}')


#Computing the confusion matrix 
cm = confusion_matrix(y_test, pred_test, labels=all_labels)

#Visualizing the confusion matrix using a heatmap
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=all_labels, yticklabels=all_labels)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
#Scaling the unseen feature data using the same scaler fitted on training data
X_unseen_scaled = scaler.transform(X_unseen)

#Predicting target values for the unseen data
y_pred_unseen = RF_best.predict(X_unseen_scaled)

#Computing classification metrics for the unseen data
acc_unseen = accuracy_score(y_unseen, y_pred_unseen)                              
precision_unseen = precision_score(y_unseen, y_pred_unseen, average='weighted', zero_division=0) 
recall_unseen = recall_score(y_unseen, y_pred_unseen, average='weighted', zero_division=0)        
f1_unseen = f1_score(y_unseen, y_pred_unseen, average='weighted', zero_division=0)                

#Printing the evaluation results
print(f'Accuracy: {acc_unseen:.4f}')
print(f'Precision: {precision_unseen:.4f}')
print(f'Recall: {recall_unseen:.4f}')
print(f'F1 Score: {f1_unseen:.4f}')

#Creating a confusion matrix for the unseen data
cm_unseen = confusion_matrix(y_unseen, y_pred_unseen)

#Generate readable class labels 
class_labels = [f"Class {i}" for i in np.unique(y_unseen)]

#Plotting the confusion matrix as a heatmap
sns.heatmap(cm_unseen, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
#Defining a function to log model evaluation metrics and hyperparameters to an Excel-file of chosen name
def log_results(model_name, params, accuracy_train, precision_train, recall_train, f1_train,
                accuracy_test, precision_test, recall_test, f1_test,
                accuracy_unseen, precision_unseen, recall_unseen, f1_unseen,
                filename="Results.xlsx"):
    """ Logs classification model results in separate sheets within the same Excel file, ensuring appending works correctly. """

    #Creating DataFrame for this model run
    result = pd.DataFrame([{
        **params, 
        "Accuracy_Train": accuracy_train,
        "Precision_Train": precision_train,
        "Recall_Train": recall_train,
        "F1_Train": f1_train,
        "Accuracy_Test": accuracy_test,
        "Precision_Test": precision_test,
        "Recall_Test": recall_test,
        "F1_Test": f1_test,
        "Accuracy_Unseen": accuracy_unseen,
        "Precision_Unseen": precision_unseen,
        "Recall_Unseen": recall_unseen,
        "F1_Unseen": f1_unseen
    }])

    #Introducing a short delay to avoid file conflicts if running in multiple notebooks
    time.sleep(1)

    #Checking if the file exists
    file_exists = os.path.exists(filename)

    if not file_exists:
        #If file doesn't exist, creating a new one
        with pd.ExcelWriter(filename, engine="openpyxl", mode="w") as writer:
            result.to_excel(writer, sheet_name=model_name, index=False)
    else:
        #If file exists, loading it properly before appending
        try:
            with pd.ExcelWriter(filename, engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
                #Reading existing sheet 
                try:
                    existing_df = pd.read_excel(filename, sheet_name=model_name, engine="openpyxl")
                    df_combined = pd.concat([existing_df, result], ignore_index=True)
                except (FileNotFoundError, ValueError):
                    df_combined = result  #If sheet does not exist, creating it

                #Saveing results, ensuring correct appending
                df_combined.to_excel(writer, sheet_name=model_name, index=False)

        except PermissionError:
            #Printing error to warn user
            print(f"Error: Close the Excel file ({filename}) before running the script again.")

    #Printing a confirmation to ensure user results are logged
    print(f"Logged results for {model_name}: Train Accuracy={accuracy_train:.4f}, Test Accuracy={accuracy_test:.4f}, Unseen Accuracy={accuracy_unseen:.4f}")

In [ ]:
#Using the function to log results from RF classifier model
log_results(
    model_name="RF_stat",
    params=best_params,

    accuracy_train=acc_train,
    precision_train=precision_train,
    recall_train=recall_train,
    f1_train=f1_train,

    accuracy_test=acc_test,
    precision_test=precision_test,
    recall_test=recall_test,
    f1_test=f1_test,

    accuracy_unseen=acc_unseen,
    precision_unseen=precision_unseen,
    recall_unseen=recall_unseen,
    f1_unseen=f1_unseen
)
